 # Actor Pool


In [1]:
import ray
from ray.util import ActorPool


@ray.remote
class Actor:
    def double(self, n):
        return n * 2


a1, a2 = Actor.remote(), Actor.remote()
pool = ActorPool([a1, a2])

# pool.map(..) returns a Python generator object ActorPool.map
# lambda 参数1, 参数2: 返回值
# pool.map(task_function, iterable)
gen = pool.map(lambda a, v: a.double.remote(v), [1, 2, 3, 4])

print(list(gen))  # result: [2, 4, 6, 8]

2024-12-14 14:35:50,196	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8267 


[2, 4, 6, 8]


## Message passing using Ray Queue

In [7]:
import ray
from ray.util.queue import Queue, Empty

ray.shutdown()
ray.init()
# You can pass this object around to different tasks/actors
queue = Queue(maxsize=100)


@ray.remote
def consumer(id, queue):
    try:
        while True:
            next_item = queue.get(block=True, timeout=1)
            print(f"consumer {id} got work {next_item}")
    except Empty:
        pass


[queue.put(i) for i in range(10)]
print("Put work 1 - 10 to queue...")

consumers = [consumer.remote(id, queue) for id in range(2)]
ray.get(consumers)

2024-12-14 14:44:07,728	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8267 


Put work 1 - 10 to queue...
(consumer pid=6669) consumer 0 got work 0
(consumer pid=6669) consumer 0 got work 2
(consumer pid=6669) consumer 0 got work 4
(consumer pid=6669) consumer 0 got work 6
(consumer pid=6669) consumer 0 got work 9


[None, None]